In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -qU "python-gdcm" pydicom pylibjpeg "opencv-python-headless"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.7 MB/s eta 0:00:00


In [3]:
# kaggle API로 데이터를 다운로드합니다.
# kaggle token을 업로드해야합니다 -> 발급받아야함

from google.colab import files
kaggle_token = files.upload()

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
# data 디렉토리 생성
!mkdir /content/data

# 캐글 데이터 다운로드
!kaggle competitions download -c rsna-pneumonia-detection-challenge

# 다운로드 받은 케글 데이터를 data 디렉토리로 이동
!mv rsna-pneumonia-detection-challenge.zip /content/data

# 캐글 데이터를 data 디렉토리에 압축풀기
!unzip -q /content/data/rsna-pneumonia-detection-challenge.zip -d /content/data

100% 3.66G/3.66G [01:49<00:00, 37.0MB/s]
100% 3.66G/3.66G [01:49<00:00, 36.0MB/s]


In [5]:
# YOLOv5 깃클론

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-129-gb54fd0a Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.1/166.8 GB disk)


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pydicom
import os
import glob
from tqdm import tqdm
import zipfile
from sklearn.model_selection import train_test_split
import yaml
import torch

In [7]:
train = pd.read_csv('/content/data/stage_2_train_labels.csv')
train

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [8]:
# train csv에 각 이미지 경로를 추가함
image_root = '/content/data/stage_2_train_images'
paths = []
for k in tqdm(range(len(train))):
  row = train.iloc[k, :]
  path = os.path.join(image_root, row['patientId']) + '.dcm'
  paths.append(path)

train['path'] = paths

100%|██████████| 30227/30227 [00:08<00:00, 3679.07it/s]


In [9]:
train['patientId'].duplicated().sum(), len(train)

(3543, 30227)

In [10]:
# 데이터프레임을 넣으면 YOLO 형태의 txt가 생성되는 함수

def get_YOLO_txt(df):
  df = df.sort_values(by='patientId')

  for k in tqdm(range(len(df))):
    row = df.iloc[k, :]

    if row['Target'] == 0:
      with open(os.path.join('/content/yolov5/data/labels', row['patientId'] + '.txt'), 'a') as f:
        f.write('')

    else:
      x_min, y_min, w, h = row['x'], row['y'], row['width'], row['height']
      dw, dh = pydicom.dcmread(row['path']).pixel_array.astype(np.float32).shape
      dw, dh = 1/dw, 1/dh

      x_center = (x_min + w/2) * dw
      y_center = (y_min + h/2) * dh
      w = w * dw
      h = h * dh

      line = '0' + ' ' + ' '.join(map(str, [x_center, y_center, w, h]))

      with open(os.path.join('/content/yolov5/data/labels', row['patientId'] + '.txt'), 'a') as f:
        f.write(line + '\n')

In [11]:
# YOLO 디렉토리의 labels 채우기

!mkdir /content/yolov5/data/labels
get_YOLO_txt(train)

100%|██████████| 30227/30227 [02:54<00:00, 173.37it/s]


In [ ]:
# 경로를 입력하면 dicom을 png로 바꾸는 함수
def dcm_to_png(dcm_path, destination):
    """
    Convert DICOM (.dcm) files to PNG (.png) files and save them in the specified destination folder.
    
    Parameters:
    dcm_path (list): List of file paths for the input DICOM files
    destination (str): Path to the output directory where the PNG files will be saved
    
    Returns:
    None
    """
    # Make sure destination folder exists
    if not os.path.exists(destination):
        os.makedirs(destination)
        
    # Iterate over DICOM files and convert to PNG
    for path in tqdm(dcm_path):
        # Load DICOM file
        dcm = pydicom.dcmread(path)
        
        # Convert pixel data to uint8 and normalize
        img = (dcm.pixel_array / np.max(dcm.pixel_array) * 255).astype(np.uint8)
        
        # Save as PNG file
        filename = os.path.basename(path).replace('.dcm', '.png')
        filepath = os.path.join(destination, filename)
        cv2.imwrite(filepath, img)

In [ ]:
# dcm_paths = glob.glob('/content/data/stage_2_train_images/*.dcm')
# destination_path = '/content/yolov5/data/images'
# dcm_to_png(dcm_paths, destination_path)

100%|██████████| 26684/26684 [19:24<00:00, 22.91it/s]


In [12]:
!unzip -q /content/drive/MyDrive/P_images.zip -d /content/yolov5/data/images

In [ ]:
# png 변환이 오래걸려서, 구글드라이브에 압축해서 저장해놓으려고함

def compress_files(input_directory, output_zipfile):
    # input_directory 내의 모든 파일 목록 가져오기
    files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]
    num_files = len(files)

    # zip 파일 열기
    with zipfile.ZipFile(output_zipfile, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        # 모든 파일 압축하기
        for file in tqdm(files, desc='Compressing files'):
            file_path = os.path.join(input_directory, file)
            zipf.write(file_path, arcname=file)

    print(f'{num_files} files compressed successfully into {output_zipfile}')

# Example usage
input_directory = '/content/yolov5/data/images'
output_zipfile = '/content/drive/MyDrive/P_images.zip'
compress_files(input_directory, output_zipfile)

Compressing files: 100%|██████████| 26684/26684 [10:02<00:00, 44.27it/s]


26684 files compressed successfully into /content/drive/MyDrive/P_images.zip


In [13]:
# train, valid
images_paths = glob.glob('/content/yolov5/data/images/*.png')

train_path, valid_path = train_test_split(images_paths,
                                          test_size=0.1,
                                          random_state=777,
                                          shuffle=True)

with open('/content/train.txt', 'w') as f:
  f.write('\n'.join(train_path) + '\n')

with open('/content/valid.txt', 'w') as f:
  f.write('\n'.join(valid_path) + '\n')

In [14]:
# train.txt, val.txt 파일 경로
train_file = '/content/train.txt'
val_file = '/content/valid.txt'

# 클래스 이름 리스트
classes = ['pneumonia']

# data.yaml 파일 경로
data_file = '/content/data.yaml'

# data.yaml 파일 생성
data = dict(
    train=train_file,
    val=val_file,
    nc=len(classes),
    names=classes
)

# 한글 문자열 지원을 위한 설정
yaml.add_representer(str, lambda dumper, data: dumper.represent_scalar('tag:yaml.org,2002:str', data, style='"'))

with open(data_file, 'w', encoding='UTF-8') as f:
    yaml.dump(data, f, allow_unicode=True)

In [ ]:
!python train.py --img 416 --batch 16 --epochs 5 --data /content/data.yaml --cfg /content/yolov5/models/yolov5m.yaml --weights yolov5m.pt --name /content/drive/MyDrive/Pneumonia_model1_5peochs

# inference 

In [ ]:
dcm_paths = glob.glob('/content/data/stage_2_test_images/*.dcm')
destination_path = '/content/data/test_images'
dcm_to_png(dcm_paths, destination_path)

 25%|██▍       | 736/3000 [00:37<01:22, 27.31it/s]

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/drive/MyDrive/Pneumonia_model1_10peochs/weights/best.pt')
test = pd.read_csv('/content/data/stage_2_sample_submission.csv')

In [ ]:
# model을 입력하면 

def inference(df):
  predicts = []

  for k in tqdm(range(len(df))):
    row = df.iloc[k, :]
    image_name = row['patientId']
    test_image_path = os.path.join('/content/data/test_images', image_name + '.png')
    image = cv2.cvtColor(cv2.imread(test_image_path), cv2.COLOR_BGR2RGB)
    
    predict = model(image)
    bboxes = ''
    for pred in predict:
      classes, confidence, x, y, w, h = pred
      bbox = ' '.join(map(str, [confidence, x, y, w, h])) + ' '
      bboxes += bbox
    bboxes = bboxes.rstrip()

    line = {'patientId':image_name,
            'PredictionString':bboxes}
    predicts.append(line)

  df_pred = pd.DataFrame(columns=['patientId', 'PredictionString'], data=predicts)
  return df_pred

In [ ]:
test_image_path = '/content/data/stage_2_test_images'

paths = []
dws = []
dhs = []
for k in tqdm(range(len(test))):
  row = test.iloc[k, :]
  path = os.path.join(test_image_path, row['patientId'] + '.dcm')
  paths.append(path)

  image = pydicom.dcmread(path).pixel_array.astype(np.float32)
  dw, dh = image.shape
  dws.append(dw)
  dhs.append(dh)

test['path'] = path
test['width'] = dws
test['height'] = dhs

In [ ]:
test['height'].unique()

array([1024])

In [ ]:
!python detect.py --weights /content/drive/MyDrive/Pneumonia_model1_5peochs/weights/best.pt --img 416 --conf 0.25 --source /content/data/test_images --save-txt

In [ ]:
len(os.listdir('/content/yolov5/runs/detect/exp/labels'))

880

In [51]:
# detection.py로 생성된 txt를 csv로 변환합니다
# txt의 기본 디렉토리는 /content/yolov5/runs/detect/exp/labels

import os
import csv

def txt_to_csv(directory_path, output_csv_path):
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["patientId", "PredictionString"])

        for file_name in os.listdir(directory_path):
            if file_name.endswith(".txt"):
                file_path = os.path.join(directory_path, file_name)

                with open(file_path, 'r', encoding='utf-8') as txtfile:
                    content = txtfile.read()
                    csv_writer.writerow([file_name, content])

    print(f"CSV file created at: {output_csv_path}")

# Example usage:
txt_directory_path = "/content/yolov5/yolov5/runs/detect/exp/labels"
output_csv_path = "/content/output.csv"
txt_to_csv(txt_directory_path, output_csv_path)

CSV file created at: /content/output.csv


In [52]:

def ratio2int(string):
  strings = string.split('\n')
  img_width = 1024
  img_height = 1024

  ls = ''
  for point in strings:
    if point == '':
      continue

    point = point.split()
    point = [float(i) for i in point]

    cls, x_center, y_center, width, height = point

    # Convert relative coordinates to pixel-based coordinates
    x_center_pixel = x_center * img_width
    y_center_pixel = y_center * img_height
    width_pixel = width * img_width
    height_pixel = height * img_height

    # Calculate the pixel-based bounding box coordinates
    cls = int(cls)
    x_min_pixel = int(round(x_center_pixel - (width_pixel / 2), 0))
    y_min_pixel = int(round(y_center_pixel - (height_pixel / 2), 0))
    width_pixel = int(round(width_pixel, 0))
    height_pixel = int(round(height_pixel, 0))

    ans = ' '.join(map(str, [cls, x_min_pixel, y_min_pixel, width_pixel, height_pixel])) + ' '
    ls += ans
  ls = ls.rstrip()
  return ls

In [54]:
output = pd.read_csv('/content/output.csv')
output['patientId'] = output['patientId'].apply(lambda x: x.split('.')[0])
output['PredictionString'] = output['PredictionString'].apply(ratio2int)
output

,patientId,PredictionString
0,02431455-505f-4ece-bac3-b4f494c65366,0 168 356 251 334
1,c1ef6724-f95f-40f1-b25b-de806d9bc39d,0 220 306 242 426
2,2017c617-7593-4996-9faf-3db972b7e7d9,0 155 241 258 672 0 630 382 261 550
3,2f6ca235-d1f3-4a84-9f4a-645f9b536672,0 601 333 255 556 0 163 567 238 345
4,1bdf99c5-f857-4f7f-bd32-3c75921b9799,0 210 526 251 299
...,...,...
875,0114657d-6647-492a-9494-110ba43e4135,0 249 278 215 442
876,252386b6-2a6c-4e45-98a7-9af73a1f9b13,0 552 218 261 483 0 98 536 260 266 0 589 307 2...
877,0f322913-eeda-4f18-b112-a9da608c5a3a,0 632 294 228 477
878,2d2633d2-b5a5-4f56-901e-6599355cebd4,0 524 391 215 289


In [64]:
test = pd.read_csv('/content/data/stage_2_sample_submission.csv')
test['PredictionString'] = test['PredictionString'].apply(lambda x: '')
test

,patientId,PredictionString
0,0000a175-0e68-4ca4-b1af-167204a7e0bc,
1,0005d3cc-3c3f-40b9-93c3-46231c3eb813,
2,000686d7-f4fc-448d-97a0-44fa9c5d3aa6,
3,000e3a7d-c0ca-4349-bb26-5af2d8993c3d,
4,00100a24-854d-423d-a092-edcf6179e061,
...,...,...
2995,c1e88810-9e4e-4f39-9306-8d314bfc1ff1,
2996,c1ec035b-377b-416c-a281-f868b7c9b6c3,
2997,c1ef5b66-0fd7-49d1-ae6b-5af84929414b,
2998,c1ef6724-f95f-40f1-b25b-de806d9bc39d,


In [62]:
def merge_to_answer(df1, df2):
  merged_df = df1.merge(df2, on='patientId', how='left', suffixes=('', '_y'))
  merged_df['PredictionString'] = merged_df.apply(
    lambda row: row['PredictionString_y'] if pd.notna(row['PredictionString_y']) else row['PredictionString'],
    axis=1)
  merged_df.drop(columns=['PredictionString_y'], inplace=True)
  return merged_df

In [78]:
merge_to_answer(test, output).to_csv('/content/submission.csv', index=False)